# Paper reproduction: Communication-Efficient Distributed SGD using Preamble-based Random Access by Choi Available at https://arxiv.org/abs/2105.09427

Straighfoward reproduction without classes

## Imports

In [8]:
# Import needed libraries
import numpy as np # arrays
import matplotlib.pyplot as plt #ploting
import scipy.optimize as sciopt    # linprog
from scipy.optimize import nnls # NNLS SOLVER

## Auxiliary functions

Construct the codebook as a scaled cross polytope:
$$ \cal{C}_{cp} = \{ \pm \textrm{R}e_l : \{1,...,\textrm{L}\}\} $$
with $ R = \sqrt{L} $, with $L$ being the block length.

In [9]:
def construct_codebook(blockLength):
    # Construct the codebook as a scaled cross polytope
    R = np.sqrt(blockLength)
    codebook = np.concatenate(((np.eye(blockLength)*R),(np.eye(blockLength)*(-R))),axis=0) + 0 # add zero to fix -0.0 issue
    return codebook

In [10]:
def quantize(codebook,numBlocks,blockLength,gradient,Vmax):
        
        codebook = codebook
        
        prob = np.zeros(numBlocks)
        norms = np.zeros(numBlocks)
        c_idx = np.zeros(numBlocks, int)
        
        for block in range(numBlocks):
            
            begin = block*blockLength
            end = block*blockLength+blockLength
            
            v_d = gradient[begin:end]
            v_til = (v_d)/np.linalg.norm(v_d)
            norms[block] = np.linalg.norm(v_d)
            
            if(norms[block] > Vmax):
                
                v_d = v_d*(Vmax/norms[block])
                norms[block] = Vmax
                gradient[begin:end] = v_d

            # solving the linear system
            M = codebook.shape[0]
            A = np.concatenate((codebook.T,np.ones((1,M))),axis=0)
            b = np.concatenate((v_til,np.ones(1)),axis=0)
            
            
            a_vec = nnls(A, b)[0] # Using Non-negative Least Squares
            ## scipy.optimize.nnls solves the KKT (Karush-Kuhn-Tucker) conditions for the non-negative least squares problem.

            # print(np.sum(np.abs(v_til-a_vec@codebook)**2))
            # input()
            
            
            c_idx[block] = np.random.choice(M, p=a_vec) # use a_vec as probabilities for choosing the codeword
            # print(c_idx[block])
            # a_probs[block] = a_vec[c_idx[block]]
            prob[block] = norms[block]/Vmax
            
        return prob, norms, c_idx, gradient # returns the norms and the indexs of the codewords

In [11]:
def RA_mix(prob, qidx, codebook,blockLength, numBlocks):
    
    grad = np.zeros(L)
    for k in range(K):
        
        for block in range(numBlocks):
            
            begin = block*blockLength
            end = block*blockLength+blockLength
            p=0
            if np.random.normal() < prob[k,block]:
                p = 1
            qi = qidx[k,block]
            grad[begin:end] += (p+np.random.normal())*codebook[qi]
            
    
    grad = grad/K
    
    return grad

In [12]:
def RA_mix(prob, qidx, codebook,blockLength, numBlocks):
    
    P=1
    snr = 10
    M = codebook.shape[0]
    K = prob.shape[0]
    grad = np.zeros(L)
    noise = 1/snr
    z = np.zeros((blockLength,M)) #zm
    N = 100
        
    for block in range(numBlocks):

        bk = np.zeros(K)
        for k in range(K):
            if np.random.normal() < probs[k,block]:
                bk[k] = 1

        a = np.zeros(blockLength)

        for m in range(M):
            Id = np.eye(blockLength)
            SigmaM = sum(bk[np.where(qidx==m)[0]]+noise)*Id
            # print(SigmaM.shape)
            zm = np.sqrt(0.5)*(np.random.multivariate_normal(mean=np.zeros(blockLength),cov=SigmaM)+np.random.multivariate_normal(mean=np.zeros(blockLength),cov=SigmaM)*(1j))
            # print(zm.shape)
            # print(codebook[m].shape)
            a += (np.linalg.norm(zm)**2)*(codebook[m]/N)

        begin = block*blockLength
        end = block*blockLength+blockLength

        grad[begin:end] = a*Vmax
    
    grad = grad/K
    
    return grad

In [263]:
begin = block*blockLength
end = block*blockLength+blockLength
bk=0
if np.random.normal() < prob[k,block]:
    bk = 1
qi = qidx[k,block]
grad[begin:end] += (bk+noise[qi])*codebook[qi]/100

NameError: name 'block' is not defined

In [116]:
bk = np.zeros((probs.shape))
for k in range(K):
    if np.random.normal() < probs[k]:
        bk[k] = 1

In [139]:
for m in M:
    SigmaM = np.eye(L)
    bk[np.where(qidx==m)[0]]+noise

TypeError: 'int' object is not iterable

In [210]:
a = np.zeros(L)
N = 100
for m in range(M):
    Id = np.eye(L)
    SigmaM = sum(bk[np.where(qidx==m)[0]]+noise)*Id
    zm = np.sqrt(0.5)*(np.random.multivariate_normal(mean=np.zeros(L),cov=SigmaM)+np.random.multivariate_normal(mean=np.zeros(L),cov=SigmaM)*(1j))
    a += (np.linalg.norm(zm)**2)*(codebook[m]/N)
    
a = a/K
print(a.shape)

(8,)


0.16793173260138178
[3.3]


In [249]:
np.sum(np.abs(true_g-a*Vmax)**2)

0.03695591511453661

In [175]:
1j

1j

In [135]:
noise=P/snr

In [163]:
zm = np.random.multivariate_normal(mean=np.zeros(L),cov=SigmaM,size=8)

print(np.cov(zm))
print(SigmaM)

[[ 6.45179644 -1.06762983 -0.43180044  0.57650261 -1.10535614  0.48920949
  -0.71387827  0.49479275]
 [-1.06762983  2.56235012  1.82514393 -0.10426963  0.33615995  1.50592642
   1.11219133 -1.50162694]
 [-0.43180044  1.82514393  2.2096894  -0.64163515  0.49509436  2.03532835
   0.58956224 -0.65182044]
 [ 0.57650261 -0.10426963 -0.64163515  1.8162775  -0.53135178 -0.66324262
  -0.99967636 -0.27457026]
 [-1.10535614  0.33615995  0.49509436 -0.53135178  3.07293123 -1.82791481
   2.6263195  -0.53102863]
 [ 0.48920949  1.50592642  2.03532835 -0.66324262 -1.82791481  5.01290826
  -1.0228731  -0.58879478]
 [-0.71387827  1.11219133  0.58956224 -0.99967636  2.6263195  -1.0228731
   4.75314002 -1.36704419]
 [ 0.49479275 -1.50162694 -0.65182044 -0.27457026 -0.53102863 -0.58879478
  -1.36704419  1.3996551 ]]
[[3.3 0.  0.  0.  0.  0.  0.  0. ]
 [0.  3.3 0.  0.  0.  0.  0.  0. ]
 [0.  0.  3.3 0.  0.  0.  0.  0. ]
 [0.  0.  0.  3.3 0.  0.  0.  0. ]
 [0.  0.  0.  0.  3.3 0.  0.  0. ]
 [0.  0.  0.  0. 

In [100]:
P=1
snr=10
n = np.random.normal(0,P/snr,codebook.shape[0])


## Definitions

In [28]:
# Definitions
K = 500 # Number of datasets distributed over K devices (i.e., 1 data set per device)
numBlocks = 10  # amount of blocks in a gradient vector (D subvectors)
blockLength = 8 # block length
L = blockLength*numBlocks # Lenght of w
u =  0.01 # step size or learning rate
T = 100 # Number of iterations
Vmax = np.sqrt(blockLength)*1.5

## Testing

In [29]:
# %%time
mcr = 100
mse = 0
mse_teo = 0
snr = 10
P = 1
N_0 = P/snr
N = 100 # Number of antennas
codebook = construct_codebook(blockLength)

for mx in range(mcr):
    true_g = np.zeros(L)
    est_g = np.zeros(L)
    gradients = np.random.normal(size = (K,L))
    probs = np.zeros((K,numBlocks))
    qidx = np.zeros((K,numBlocks),int)
    norms_v = np.zeros((K,numBlocks))

    for k in range(K):
        probs[k,:], norms_v[k,:], qidx[k,:], gradients[k,:] = quantize(codebook,numBlocks,blockLength,gradients[k,:],Vmax)
        # print(norms_v)
        # input()
    est_g = RA_mix(probs, qidx, codebook,blockLength, numBlocks)
    true_g = np.sum(gradients,axis = 0)/K

    mse +=  np.sum(np.abs(true_g-est_g)**2)/mcr
    mse_teo += (np.sum(np.sum(np.abs(blockLength*Vmax - norms_v)*norms_v))/(K**2))/mcr;
    
print(mse,mse_teo,K)

5.594878496183728 1.69768382855381 500


In [304]:
norms_v.shape

(500, 10)

In [93]:
(codebook[0:8].T@codebook[0:8])

array([[8., 0., 0., 0., 0., 0., 0., 0.],
       [0., 8., 0., 0., 0., 0., 0., 0.],
       [0., 0., 8., 0., 0., 0., 0., 0.],
       [0., 0., 0., 8., 0., 0., 0., 0.],
       [0., 0., 0., 0., 8., 0., 0., 0.],
       [0., 0., 0., 0., 0., 8., 0., 0.],
       [0., 0., 0., 0., 0., 0., 8., 0.],
       [0., 0., 0., 0., 0., 0., 0., 8.]])

In [92]:
codebook[0:8]

array([[2.82842712, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 2.82842712, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 2.82842712, 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 2.82842712, 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 2.82842712,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        2.82842712, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 2.82842712, 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.82842712]])

In [81]:
np.dot(codebook.T, codebook, out=None)

array([[16.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 16.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 16.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0., 16.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0., 16.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 16.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 16.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 16.]])

In [95]:
np.linalg.norm(codebook[9])

2.8284271247461903

In [97]:
np.sqrt(8)

2.8284271247461903